In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
path = "/usr/local/bin/chromedriver"
s=Service(path)

In [52]:
#fire up the browser
browser = webdriver.Chrome(service=s)

In [53]:
#go to amazon
link = 'https://www.amazon.com/'
browser.get(link)

In [54]:
#the product name
product = 'Over-Ear Headphones'

#put it in the search bar
browser.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]').send_keys(product)
time.sleep(4)

#press enter
browser.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]').send_keys(Keys.ENTER)
time.sleep(4)


In [55]:
# a helper function to check if the button exists on the page
def check_exists_by_xpath(xpath):
    try:
        browser.find_element(By.XPATH,xpath)
    except NoSuchElementException:
        return False
    return True

In [56]:
#click the tab 1 star and above
browser.find_element(By.XPATH, '//*[@id="p_72/1248882011"]/span/a').click()

In [57]:
#store the url of products rated as 1 star and above
url2 = browser.current_url
print(url2)

https://www.amazon.com/s?k=Over-Ear+Headphones&rh=n%3A172282%2Cp_72%3A1248882011&dc&ds=v1%3AxhBXM69GOIMyIy8fipb39k%2FAl1bp0Ip3GvUEXFVRZ3U&crid=X8TI5A4VHEDY&qid=1671167589&rnid=1248877011&sprefix=over-ear+headphones%2Caps%2C2069&ref=sr_nr_p_72_4


In [58]:
list_of_lists = list()

#set implicit wait time
browser.implicitly_wait(10)

#5 pages of product review, each page has 10 reviews, ideally 50 reviews for each product, 1500 reviews in total
k_pages = 5

#going thru all products
for n in range(30):
    print(f'Going thru product {n+1}')
    time.sleep(2)
    browser.get(url2)
    time.sleep(2)
    
    #30 products stored in a list, we are going to url2 and click on each product using a for loop
    list_products = browser.find_elements(By.XPATH, '//div[@class = "s-product-image-container aok-relative s-image-overlay-grey s-text-center s-padding-left-small s-padding-right-small s-flex-expand-height"]')
    time.sleep(2)
    
    #click the nth product
    list_products[n].click()
    time.sleep(2)
    
    #click to see all reviews
    browser.find_elements(By.XPATH, '//a[@class="a-link-emphasis a-text-bold"]')[-1].click()
    time.sleep(2)

    # go thru all review pages
    for i in range(k_pages):
        list_author = list()
        list_review = list()
        list_date = list()
        list_rating = list()

        #parse to a soup
        page_source = browser.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        # find all 10 reviews on one page
        reviews_content = soup.find_all('div', class_='a-section review aok-relative')
        time.sleep(3)

        # for each review, find corresponding author, content, date and rating
        for review in reviews_content:
            author = review.find('div', class_='a-profile-content').text
            list_author.append(author)

            txt = review.find('div', class_='a-row a-spacing-small review-data').text
            list_review.append(txt)

            date = review.find('span', class_='a-size-base a-color-secondary review-date').text.split()[-3:]
            list_date.append(date)

            rating_line = review.find('span', class_='a-icon-alt')
            rating = rating_line.text.split()[0]
            list_rating.append(rating)

        for j in range(len(list_rating)):
                list_info = list()
                list_info.append(list_review[j])
                list_info.append(list_rating[j])
                list_info.append(list_author[j])
                list_info.append(list_date[j])
                list_of_lists.append(list_info)

        time.sleep(2)
        if check_exists_by_xpath('//li[@class="a-disabled a-last"]'):
            print(f'The review of this product ends at page {i+1}')
            break
            time.sleep(2)
        else:
            browser.find_element(By.XPATH,'//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a').click()
            time.sleep(2)

The review of this product ends at page 2


In [59]:
print(len(list_of_lists))


1467


In [61]:
#dataframe with column names
col_name = ['Review', 'Rating', 'Author', 'Date']
df = pd.DataFrame(list_of_lists,columns=col_name)

In [62]:
%pwd

'/Users/yijiexu/Documents/WebAnalytics/Group Project'

In [63]:
#export the df to csv file
df.to_csv('WA_Amazon_RawData.csv')

In [67]:
df.iloc[1]['Review']

'\nIntroduction----------------------------- I love testing headphones, In-ear-monitors, earphones, earbuds, earwax (jkjk)...ya know all of that stuff. Recently I picked up a pair of Philips SHP9500S (Note the "S" refers to the new version as the original 9500 was discontinued years ago). These were originally intended as a replacement for my broken gaming headphones, but upon receiving them and testing them on music I was extremely surprised with their sonic performance (or quality of their sound). At $70 these are what I would consider to be relatively "budget friendly" headphones. What surprised me the most was that even at that price, they competed equally with headphones/earphones/IEMs upwards of 3x that price point. Now I understand that these are Open-Back headphones and aren\'t suitable for everyone\'s needs (more on that later) and that the sound signature/style of sound is not traditional to what people are accustomed to hearing (again...more on this later). So without furthe

In [25]:
browser.quit()